<a href="https://colab.research.google.com/github/aderdouri/ql_web_app/blob/master/ql_notebooks/extensibleoptions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import QuantLib as ql
import unittest
import math

# Helper utilities (assuming flat_rate_py, flat_vol_py are defined as in previous examples)
def flat_rate_py(evaluation_date_or_quote, forward_rate_or_dc, day_counter_or_none=None):
    if isinstance(evaluation_date_or_quote, ql.Date):
        evaluation_date = evaluation_date_or_quote
        forward_rate_obj = forward_rate_or_dc
        day_counter = day_counter_or_none
    else:
        evaluation_date = ql.Settings.instance().evaluationDate
        forward_rate_obj = evaluation_date_or_quote
        day_counter = forward_rate_or_dc

    if isinstance(forward_rate_obj, ql.Quote):
        quote_handle = ql.QuoteHandle(forward_rate_obj)
    elif isinstance(forward_rate_obj, float):
        quote_handle = ql.QuoteHandle(ql.SimpleQuote(forward_rate_obj))
    else:
        quote_handle = forward_rate_obj
    return ql.FlatForward(evaluation_date, quote_handle, day_counter)

def flat_vol_py(evaluation_date_or_quote, vol_level_or_dc, day_counter_or_none=None):
    if isinstance(evaluation_date_or_quote, ql.Date):
        evaluation_date = evaluation_date_or_quote
        vol_level_obj = vol_level_or_dc
        day_counter = day_counter_or_none
    else:
        evaluation_date = ql.Settings.instance().evaluationDate
        vol_level_obj = evaluation_date_or_quote
        day_counter = vol_level_or_dc

    if isinstance(vol_level_obj, ql.Quote):
        vol_quote_handle = ql.QuoteHandle(vol_level_obj)
    elif isinstance(vol_level_obj, float):
        vol_quote_handle = ql.QuoteHandle(ql.SimpleQuote(vol_level_obj))
    else:
        vol_quote_handle = vol_level_obj
    return ql.BlackConstantVol(evaluation_date, ql.NullCalendar(), vol_quote_handle, day_counter)


class ExtensibleOptionsTests(unittest.TestCase):

    def setUp(self):
        self.saved_eval_date = ql.Settings.instance().evaluationDate
        # Set a default evaluation date for tests
        self.today = ql.Date(15, ql.May, 2007) # Arbitrary fixed date
        ql.Settings.instance().evaluationDate = self.today

        self.dc = ql.Actual360()

    def tearDown(self):
        ql.Settings.instance().evaluationDate = self.saved_eval_date

    def test_analytic_holder_extensible_option_engine(self):
        print("Testing analytic engine for holder-extensible option...")

        current_eval_date = ql.Settings.instance().evaluationDate # Use date from setUp

        option_type = ql.Option.Call
        strike1 = 100.0
        strike2 = 105.0
        ex_date1 = current_eval_date + ql.Period(180, ql.Days)
        ex_date2 = current_eval_date + ql.Period(270, ql.Days) # Maturity of the extended option
        premium = 1.0 # Premium to extend

        spot_q = ql.SimpleQuote(100.0)
        q_rate_q = ql.SimpleQuote(0.0)
        r_rate_q = ql.SimpleQuote(0.08)
        vol_q = ql.SimpleQuote(0.25)

        underlying_h = ql.QuoteHandle(spot_q)
        dividend_ts_h = ql.YieldTermStructureHandle(flat_rate_py(current_eval_date, q_rate_q, self.dc))
        risk_free_ts_h = ql.YieldTermStructureHandle(flat_rate_py(current_eval_date, r_rate_q, self.dc))
        black_vol_ts_h = ql.BlackVolTermStructureHandle(flat_vol_py(current_eval_date, vol_q, self.dc))

        payoff = ql.PlainVanillaPayoff(option_type, strike1)
        exercise = ql.EuropeanExercise(ex_date1)

        # HolderExtensibleOption(type, premium, exDate2, strike2, payoff, exercise)
        # type: Option::Type of the option if extended
        # premium: premium to be paid to extend
        # exDate2: maturity of the extended option
        # strike2: strike of the extended option
        # payoff: payoff of the original option
        # exercise: exercise of the original option
        option = ql.HolderExtensibleOption(option_type, premium, ex_date2, strike2, payoff, exercise)

        process = ql.BlackScholesMertonProcess(underlying_h, dividend_ts_h, risk_free_ts_h, black_vol_ts_h)
        engine = ql.AnalyticHolderExtensibleOptionEngine(process)
        option.setPricingEngine(engine)

        calculated_npv = option.NPV()
        expected_npv = 9.4233
        tolerance = 1e-4

        self.assertAlmostEqual(calculated_npv, expected_npv, delta=tolerance,
                               msg=f"Holder-extensible option NPV mismatch: "
                                   f"Expected {expected_npv}, Got {calculated_npv}")

    def test_analytic_writer_extensible_option_engine(self):
        print("Testing analytic engine for writer-extensible option...")

        current_eval_date = ql.Settings.instance().evaluationDate

        option_type = ql.Option.Call # Both options are calls in this test
        strike1 = 90.0 # Strike of initial option
        strike2 = 82.0 # Strike of option if extended
        ex_date1 = current_eval_date + ql.Period(180, ql.Days) # Maturity of initial option
        ex_date2 = current_eval_date + ql.Period(270, ql.Days) # Maturity of extended option

        spot_q = ql.SimpleQuote(80.0)
        q_rate_q = ql.SimpleQuote(0.0)
        r_rate_q = ql.SimpleQuote(0.10)
        vol_q = ql.SimpleQuote(0.30)

        underlying_h = ql.QuoteHandle(spot_q)
        # In C++, these TS are ext::shared_ptr, not Handles.
        # The Process takes Handles, so ensure these are wrapped if not already.
        dividend_ts = flat_rate_py(current_eval_date, q_rate_q, self.dc)
        risk_free_ts = flat_rate_py(current_eval_date, r_rate_q, self.dc)
        black_vol_ts = flat_vol_py(current_eval_date, vol_q, self.dc)

        dividend_ts_h = ql.YieldTermStructureHandle(dividend_ts)
        risk_free_ts_h = ql.YieldTermStructureHandle(risk_free_ts)
        black_vol_ts_h = ql.BlackVolTermStructureHandle(black_vol_ts)

        # C++ test uses GeneralizedBlackScholesProcess, Python uses BlackScholesMertonProcess usually
        # If GeneralizedBlackScholesProcess is specifically needed and wrapped, use that.
        # Otherwise, BlackScholesMertonProcess is the common one.
        process = ql.BlackScholesMertonProcess(underlying_h, dividend_ts_h, risk_free_ts_h, black_vol_ts_h)
        engine = ql.AnalyticWriterExtensibleOptionEngine(process)

        payoff1 = ql.PlainVanillaPayoff(option_type, strike1)
        exercise1 = ql.EuropeanExercise(ex_date1)
        payoff2 = ql.PlainVanillaPayoff(option_type, strike2) # Same type as option1 in this test
        exercise2 = ql.EuropeanExercise(ex_date2)

        # WriterExtensibleOption(payoff1, exercise1, payoff2, exercise2)
        # payoff1, exercise1: define the initial short option
        # payoff2, exercise2: define the new short option if writer extends
        option = ql.WriterExtensibleOption(payoff1, exercise1, payoff2, exercise2)
        option.setPricingEngine(engine)

        calculated_npv = option.NPV()
        expected_npv = 6.8238
        tolerance = 1e-4

        self.assertAlmostEqual(calculated_npv, expected_npv, delta=tolerance,
                               msg=f"Writer-extensible option NPV mismatch: "
                                   f"Expected {expected_npv}, Got {calculated_npv}")


if __name__ == '__main__':
    print("Testing QuantLib " + ql.__version__)
    unittest.main(argv=['first-arg-is-ignored'], exit=False)